# Pipeline de Ingestão CDC - Upcell

Este notebook implementa o pipeline de ingestão de dados CDC (Change Data Capture) do S3 para o Bronze no Databricks.

## Objetivo
- Full-load: Carga inicial completa das tabelas
- CDC: Ingestão incremental com operações Insert, Update e Delete
- Delta Lake: Merge atômico na camada Bronze

## Requisitos
- Tabelas no S3: `s3://meudatalake-raw/upcell/`
- Catálogo: `bronze.upcell`
- Coluna de controle: `DtAtualizacao` (presente em todos os arquivos)

## 1️⃣ Importações e Setup

In [0]:
import delta
def table_exists(catalog, database, table):
    count = (spark.sql(f"SHOW TABLES IN `{catalog}`.`{database}`")
               .filter(f"database = '{database}' AND tableName = '{table}'")
               .count())
    return count == 1

In [0]:
catalog = "bronze"
schema = "upcell"
tablename = dbutils.widgets.get("tablename")
id_field = dbutils.widgets.get("id_field")
timefield = dbutils.widgets.get("timefield")

## 1. Importações e Setup

In [0]:
# Carga inicial: Cria tabela Delta a partir do full-load
if not table_exists(catalog, schema, tablename):
    print(f"Tabela {catalog}.{schema}.{tablename} NÃO existe. Criando a partir do full-load...")

    # Lê full-load (já tem DtAtualizacao!)
    df_full = spark.read.format("parquet").load(f"/Volumes/raw/upcell/full-load/{tablename}")
    print(f"Total de registros no full-load: {df_full.count():,}")

    # Cria tabela Delta
    (df_full.coalesce(1)
            .write
            .format("delta")
            .mode("overwrite")
            .saveAsTable(f"{catalog}.{schema}.{tablename}"))
    print(f"Tabela {catalog}.{schema}.{tablename} criada com sucesso!")
else:
    print(f"Tabela {catalog}.{schema}.{tablename} já existe. Pular para o CDC merge.")

## 3. Full-Load (Carga Inicial)

Se a tabela não existe, cria a partir dos dados de full-load.

In [0]:
# Processa CDC: Filtra apenas arquivos novos e deduplica
# 1. Busca a última atualização já processada na tabela Bronze
last_processed = spark.sql(f"""
    SELECT COALESCE(MAX(DtAtualizacao), '1900-01-01') as last_dt
    FROM {catalog}.{schema}.{tablename}
""").collect()[0]['last_dt']

# 2. Lê TODOS os arquivos CDC (por enquanto)
(spark.read
    .format("parquet")
    .load(f"/Volumes/raw/upcell/cdc/{tablename}")
    .createOrReplaceTempView(f"view_{tablename}"))

# 3. Filtra apenas registros NOVOS (DtAtualizacao > última processada)
query_filter = f"""
    SELECT *  
    FROM view_{tablename}
    WHERE DtAtualizacao > '{last_processed}'
"""

df_cdc_new = spark.sql(query_filter)
total_new_records = df_cdc_new.count()

if total_new_records == 0:
    df_cdc_unique = df_cdc_new  # DataFrame vazio
else:
    # 4. Deduplica: Pega apenas o último registro de cada chave (nos dados NOVOS)
    query_dedupe = f"""
        SELECT *  
        FROM view_{tablename}
        WHERE DtAtualizacao > '{last_processed}'
        QUALIFY ROW_NUMBER() OVER(PARTITION BY {id_field} ORDER BY {timefield} DESC) = 1
    """
    df_cdc_unique = spark.sql(query_dedupe)

## 4. Processamento CDC

Carrega arquivos CDC, deduplica e prepara para o merge.

In [0]:
bronze = delta.DeltaTable.forName(spark, f"{catalog}.{schema}.{tablename}")
bronze


In [0]:
# Estatísticas antes do merge
count_before = spark.sql(f"SELECT COUNT(*) as total FROM {catalog}.{schema}.{tablename}").collect()[0]['total']
details_before = spark.sql(f"DESCRIBE DETAIL {catalog}.{schema}.{tablename}").select("numFiles", "sizeInBytes").collect()[0]
last_update_before = spark.sql(f"""
    SELECT MAX(DtAtualizacao) as ultima_atualizacao 
    FROM {catalog}.{schema}.{tablename}
""").collect()[0]['ultima_atualizacao']

In [0]:
# Merge CDC na tabela Delta Bronze
# Verifica se há dados novos para processar
if df_cdc_unique.count() == 0:
    # Define variáveis para comparação (sem mudanças)
    count_after = count_before
    details_after = details_before
    last_update_after = last_update_before
else:
    bronze = delta.DeltaTable.forName(spark, f"{catalog}.{schema}.{tablename}")
    (bronze.alias("b") 
      .merge(df_cdc_unique.alias("d"), f"b.{id_field} = d.{id_field}") 
      .whenMatchedDelete(condition = "d.op = 'D'")           # Delete se op = 'D'
      .whenMatchedUpdateAll(condition = "d.op = 'U'")        # Update se op = 'U'
      .whenNotMatchedInsertAll(condition = "d.op = 'I'")     # Insert se op = 'I'
      .execute()
    )
    # Estatísticas depois do merge
    count_after = spark.sql(f"SELECT COUNT(*) as total FROM {catalog}.{schema}.{tablename}").collect()[0]['total']
    details_after = spark.sql(f"DESCRIBE DETAIL {catalog}.{schema}.{tablename}").select("numFiles", "sizeInBytes").collect()[0]
    last_update_after = spark.sql(f"""
        SELECT MAX(DtAtualizacao) as ultima_atualizacao 
        FROM {catalog}.{schema}.{tablename}
    """).collect()[0]['ultima_atualizacao']
# Comparação antes vs depois
diff_records = count_after - count_before
diff_size = details_after['sizeInBytes'] - details_before['sizeInBytes']
diff_files = details_after['numFiles'] - details_before['numFiles']

## 5. Merge CDC na Tabela Delta

Aplica as operações de Insert, Update e Delete na camada Bronze.

In [0]:
# Validação 1: Contagem total de registros
total = spark.sql(f"SELECT COUNT(*) as total FROM {catalog}.{schema}.{tablename}").collect()[0]['total']
print(f"Total de registros na tabela Bronze: {total:,}")

# Validação 2: Verificar se DtAtualizacao está presente
sample = spark.sql(f"SELECT * FROM {catalog}.{schema}.{tablename} LIMIT 5")
print(f"Colunas da tabela: {sample.columns}")
sample.display()

In [0]:
# Validação 3: Verificar histórico de versões Delta
spark.sql(f"DESCRIBE HISTORY {catalog}.{schema}.{tablename}").select(
    "version", 
    "timestamp", 
    "operation", 
    "operationMetrics"
).display()